In [1]:
import csv
import pandas as pd
import dask.dataframe as dd
import os

In [2]:
path = os.getcdw()

In [3]:
tp = dd.read_csv(path + 'title.principals.tsv', sep = '\t')
tp = tp[['tconst', 'nconst']]
names = dd.read_csv(path + 'name.basics.tsv', sep = '\t')
names = names[['nconst', 'primaryName']]

In [5]:
titles = pd.read_csv(path + 'title.basics.tsv', sep = '\t', error_bad_lines = False)
titles.head()
titles = titles[['tconst', 'primaryTitle', 'originalTitle', 'startYear']]
titles['tfix'] = titles['primaryTitle'].str.lower()
titles['tfix'] = titles['tfix'].str.replace('[^\w\s]','')

In [75]:
wiki = pd.read_csv(path + 'wiki_movie_plots_deduped.csv')
wiki = wiki[['Release Year', 'Title', 'Plot']]
wiki['tfix'] = wiki['Title'].str.lower()
wiki['tfix'] = wiki['tfix'].str.replace('[^\w\s]','')
wiki.head()

,Release Year,Title,Plot,tfix
0,1901,Kansas Saloon Smashers,"A bartender is working at a saloon, serving dr...",kansas saloon smashers
1,1901,Love by the Light of the Moon,"The moon, painted with a smiling face hangs ov...",love by the light of the moon
2,1901,The Martyred Presidents,"The film, just over a minute long, is composed...",the martyred presidents
3,1901,"Terrible Teddy, the Grizzly King",Lasting just 61 seconds and consisting of two ...,terrible teddy the grizzly king
4,1902,Jack and the Beanstalk,The earliest known adaptation of the classic f...,jack and the beanstalk


In [76]:
titles['startYear'] = titles['startYear'].astype(str)
wiki['Release Year'] = wiki['Release Year'].astype(str)
movies_main = titles.merge(wiki, left_on=['tfix', 'startYear'], right_on=['tfix', 'Release Year'])
movies_main.head()

,tconst,primaryTitle,originalTitle,startYear,tfix,Release Year,Title,Plot
0,tt0000399,Jack and the Beanstalk,Jack and the Beanstalk,1902,jack and the beanstalk,1902,Jack and the Beanstalk,The earliest known adaptation of the classic f...
1,tt0000420,Alice in Wonderland,Alice in Wonderland,1903,alice in wonderland,1903,Alice in Wonderland,"Alice follows a large white rabbit down a ""Rab..."
2,tt0000439,The Great Train Robbery,The Great Train Robbery,1903,the great train robbery,1903,The Great Train Robbery,The film opens with two bandits breaking into ...
3,tt0000487,The Great Train Robbery,The Great Train Robbery,1903,the great train robbery,1903,The Great Train Robbery,The film opens with two bandits breaking into ...
4,tt0000488,The Land Beyond the Sunset,The Land Beyond the Sunset,1912,the land beyond the sunset,1912,The Land Beyond the Sunset,Joe is an impoverished New York newsboy who li...
...,...,...,...,...,...,...,...,...
36795,tt9399328,Ninnu Kalisaka,Ninnu Kalisaka,2009,ninnu kalisaka,2009,Ninnu Kalisaka,The story is a tale of two young pairs of love...
36796,tt9695066,Cuckoo,Cuckoo,2002,cuckoo,2002,Cuckoo,"September 1944. Several days before Finland, a..."
36797,tt9815072,Pontianak Menjerit,Pontianak Menjerit,2005,pontianak menjerit,2005,Pontianak Menjerit,Datuk Pengiran Abdul Rahman has asked his lawy...
36798,tt9829966,Samhita,Samhita,2013,samhita,2013,Samhita,Revati Sathe is a National Award winning docum...


In [77]:
movies_main = dd.from_pandas(movies_main, npartitions=7)
movies_main = movies_main[['tconst', 'Title','startYear','Plot']]

,tconst,Title,startYear,Plot
unique,36059,26288,117,27865
count,36800,36800,36800,36800
top,tt7046404,Home,2014,"On the run from their enemy, the so-called pla..."
freq,3,74,1694,53


In [78]:
tp = tp.merge(movies_main[['tconst','Title']], on = 'tconst')

In [79]:
names['primaryName'] = names['primaryName'].str.replace(' ','') + ' '

In [80]:
tp = tp.merge(names, left_on='nconst', right_on='nconst')

In [81]:
tp = tp.groupby(tp.tconst).primaryName.sum()
tp = tp.to_frame()
movies_main = movies_main.merge(tp, on = 'tconst')

,tconst,Title,startYear,Plot,primaryName
unique,35466,26268,117,27843,32940
count,36179,36179,36179,36179,36179
top,tt4424718,Home,2014,"On the run from their enemy, the so-called pla...",ChrisStuckmann
freq,3,72,1650,52,490


In [82]:
rev = dd.read_csv(path + 'title.ratings.tsv', sep = '\t')
movies_main = movies_main.merge(rev, on = 'tconst')

,averageRating,numVotes
count,28920.000000,2.892000e+04
mean,6.306172,2.034141e+04
std,1.077664,8.043455e+04
min,1.000000,5.000000e+00
25%,6.000000,3.110000e+02
50%,6.500000,2.145000e+03
75%,7.100000,3.479375e+04
max,10.000000,2.134710e+06


In [83]:
movies_main = movies_main.compute()

(28920, 7)

In [86]:
os.remove(path + "Movies_Reviews.csv")
movies_main.to_csv(path + 'Movies_Reviews.csv', index = False)